## How to create a Deep Ensemble for MNIST

In [ ]:
import numpy as np
import tensorflow as tf

import uncertainty_wizard as uwiz

**Step 1: Downloading Preprocessing the data**

This is the same that we would do on any regular keras mnist classifier,
except that we do not have to one-hot encode the test labels, as uncertainty wizards quantifiers
will determine the winning class (one not its one-hot encoding) for us

In [ ]:
# Lets cache the train data on the file system,
# and at the same time also prepare the test data for later
_,(x_test, y_test) = tf.keras.datasets.mnist.load_data()
x_test = (x_test.astype('float32') / 255).reshape(x_test.shape[0], 28, 28, 1)


**Step 2: Creating a Stochastic Uncertainty-Wizard Model using the Sequential API**

Note that only the first line is different from doing the same in plain keras, the rest is equivalent.
We must however ensure to use at least one randomized layer (e.g. tf.keras.layers.Dropout)

In [ ]:
# Let's create an uncertainty wizard ensemble!

#   First, we define the model setup and training process
#   This is done using traditional tensorflow code

def model_creation_and_training(model_id: int):
    model = tf.keras.models.Sequential()
    model.add(tf.keras.layers.Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(28, 28, 1)))
    model.add(tf.keras.layers.Conv2D(64, (3, 3), activation='relu'))
    model.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2)))
    model.add(tf.keras.layers.Flatten())
    model.add(tf.keras.layers.Dense(128, activation='relu'))
    model.add(tf.keras.layers.Dense(10, activation='softmax'))
    model.compile(loss=tf.keras.losses.categorical_crossentropy,
              optimizer=tf.keras.optimizers.Adadelta(),
              metrics=['accuracy'])
    # Note that we set the number of epochs to just 1, to be able to run this notebook quickly
    # Set the number of epochs higher if you want to optimally train the network
    (x_train, y_train), (_,_) = tf.keras.datasets.mnist.load_data()
    x_train = (x_train.astype('float32') / 255).reshape(x_train.shape[0], 28, 28, 1)
    y_train = tf.keras.utils.to_categorical(y_train, num_classes=10)
    fit_history = model.fit(x_train, y_train, validation_split=0.1, batch_size=32, epochs=5,
                      verbose=1, callbacks=[tf.keras.callbacks.EarlyStopping(patience=2)])
    return model, fit_history.history


In [ ]:
# Now we can define and train our ensemble model.
# Here we use two models as its faster to train in this example. In practice, you may want to use more.
ensemble = uwiz.models.LazyEnsemble(num_models=2,
                                    model_save_path="/tmp/ensemble",
                                    # Colab infrastructure is relatively weak.
                                    # Thus, lets disable multiprocessing and train on the main process.
                                    # Any argument >= 1 would result in (typically more efficient) multiprocessing
                                    # on a more powerful machine
                                    default_num_processes=0)

ensemble.create(create_function=model_creation_and_training)

**Step 3: Make predictions and get the uncertainties and confidences**

In [ ]:
# Get two one-dimensional np arrays: One containing the predictions and one containing the confidences
predictions, confidences = ensemble.predict_quantified(x_test,
                                                       quantifier='mean_softmax')